 # Instalando el ambiente

1. Create un nuevo virtualenv, activalo y actualizá `pip`: `python -m venv venv && source venv/bin/activate.fish && python -m pip install --upgrade pip setuptools`,
2. Instalá *Jupyter Lab*: `python -m pip install jupyterlab`,
3. Instalá las \"Google libs\": `python -m pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib`,
4. Creá un link simbólico que se llame `credentials.json` al JSON que tiene tus credenciales bajadas,

Se crea con el objetivo de guardar todas las librerias y dependencias necesarias que requiera el proyecto. 

# Etapa 1 Extración de los datos de los pdf
En este Jupyter notebook creo los bloques con todos los ejemplos para recopilar los direrentes tipos de datos en los pdf.
A continuación menciono que no mer sirvio y porque lo descarte. 


### 1) No me sirvio utilizar PyPDF2
Es complicado de entender la información ya que me trae metadatos como la fecha de creación del archivo, autor y demás, por lo anterior concluí no me sirve para mi desarrollo. 
Intenté con otro código pero me sigue generando un error ya teniendo instalado el paquete de PdfReader

In [1]:
import PyPDF2 
import PdfReader
pdf_file_obj= open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf", "rb")
# pdf_read= PyPDF2.PdfFileReader(pdf_file_obj)
# info =pdf_read.getDocumentInfo()
# print(info)
# print(pdf_read.pages)
# print(pdf_read.pages[0].extract_text())

reader = PdfReader(pdf_file_obj)
text = reader.pages[0].extract_text()
assert "Total" in text, text

ModuleNotFoundError: No module named 'PdfReader'

In [ ]:
from PyPDF2 import PdfFileReader, PdfFileWriter,PdfFileMerger
pdf_file_obj= open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf", "rb")
template = PdfFileReader(pdf_file_obj)
output = PdfFileWriter()

for i in range(template.getNumPages()):
   page = template.getPage(i)
   print(page)
   page.mergePage(watermark.getPage(0))
   output.addPage(page)
# file = open('waterMarked_PDF.pdf', 'wb')
# output.write(file)

{'/Contents': [IndirectObject(37, 0)], '/CropBox': [0, 0, 595.32001, 841.92004], '/MediaBox': [0, 0, 595.32001, 841.92004], '/Parent': IndirectObject(2, 0), '/Resources': IndirectObject(38, 0), '/Rotate': 0, '/Type': '/Page'}


NameError: name 'watermark' is not defined

### 2) No me sirvio utilizar fitz
Intento con html pero no tiene un buen formato para extraer información. 

In [ ]:
import fitz 
doct=fitz.open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\VIMEO_SD_PRESENTADA_1BIM_2022.pdf")
pag=doct.load_page(0)
resultado=pag.search_for("Transactions ")
extraer = pag.get_textbox(resultado)
pag.get_text("xhtml")
for line in convertido:
    # print(line)
# print(convertido)
doct_html=open("documento.html","w")
doct_html.write(convertido)
doct_html.close()
print(doct.get_toc())

IndentationError: expected an indented block (2866713009.py, line 10)

### 3) Que necesito y utilizo?

 * Leer los formularios para ello debo instalar "tabula" y "tabulate" para leer las tablas de los pdfs ya que los datos estan separados por tablas.
 * También necesito instalar Pandas para analizar los datos y adicional lo requiere Tabula.
 * Numpy sirve para analizar números.(aún no se ha utilizado) 

#### 3.1 Lectura de formatos

In [4]:
from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import numpy as np
import re #  Es para la notación de cadena raw de Python para los patrones de Unicode (str)
import locale # separador de mil en US O UK

#**Formulario IvA-ARG
ruta_arg = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf"
ruta_col = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\Decla. IVA II BIM 2022 - CO10.PDF"
f_Arg = read_pdf(ruta_arg,multiple_tables=True)
f_Col_300 = read_pdf(ruta_col,multiple_tables=True)

'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.


##### 3.1.1 Lectura de todo el pdf Argentina, extracción y captura de cada item. 

In [ ]:
año_p= f_Arg[0].iloc[[1],[3]] # pendiente separar cuit y periodo fiscal 
año_p,len(año_p),type(año_p),año_p.values[0][0] # Identifico como recorrer la lineda y se accede por medio de una array
año_pf=año_p.values[0][0]
n=año_pf.split(" ")[0]
period= str(n.split("-")[0]) #ok periodo
año= int(n.split("-")[1]) #ok año


form_unido=f_Arg[0].iloc[[4],[0]] #pendiente separar formulario y razon social 
a= form_unido.values[0][0]
n_for = a.split(" ")[0] # ok formulario

razon_soc2= a.split(" ")[7]
razon_soc1= a.split(" ")[6]+" "+razon_soc2 # ok razon social


verifi= f_Arg[0].iloc[[3],[3]] #pendiente separar form, CUIT, actividad principal y N° verificador
cod_verif= verifi.values[0][0].split(" ")[1] # lo identifico, separo y accedo

#Lectura accede a la primera y segunda sección 
for fila in f_Arg[0].loc[11:].itertuples():#salo las primeras 11 filas
    print(fila[1], fila[4])



Saldo de proveedores por operaciones con empresas promovidas 0,00
Total del débito fiscal del período 4.457.547,03
Total del crédito fiscal del período 1.143.786,04
Ajuste anual del crédito fiscal por operaciones exentas AFIP 0,00
nan Contribuyente 0,00
Reducción del Saldo Técnico - Beneficio a pequeños contribuyentes Art. 12 Ley 27.618 0,00
Saldo a favor del período anterior 0,00
Impuesto facturado vinculado con exportaciones perfeccionadas o no en el período o asimilables 0,00
Saldo técnico a favor del responsable 0,00
Saldo técnico a favor de AFIP 3.313.760,99
Crédito fiscal a imputar a otros impuestos por compra de insumos de editores libros, folletos, etc. 0,00
Diferimiento F.518 0,00
Total F.515 0,00 Total F.518 0,00 Total F.517/1 0,00
Determinación de la declaración jurada mensual (incluídos los régimenes promocionales) nan
Saldo técnico a favor de AFIP 3.313.760,99
- Bonos Fiscales - Decreto N° 1.145/09 0,00
Certificados de crédito fiscal - RG 4212 - Art. 9o 0,00
Saldo técnico 

##### a) Como extraer CUIT de los pdf? (Argentina)
Debo importar "re" para buscar expresiones regulares

In [ ]:
reg = re.compile(r'[0-9]{2}-[0-9]{8}-[0-9]{1}') # Creo el expresion regular para encontrar el CUIT
id_RazonSocial = reg.search(str(f_Arg[0].iloc[[3],[3]])) # Acá eligo la celda donde se encuentra el CUIT junto con otro texto
print(id_RazonSocial.group(0)) # Imprimo el resultado CUIT

30-70892838-7


##### b) Como convertir un str a float? 
Debo de importar libreria "locale"

In [ ]:
#Saldo técnico a favor de AFIP
saldo_AFIP = f_Arg[0].iloc[[43],[3]] #Saldo de impuesto a favor de AFIP
sal_entid = saldo_AFIP.values[0][0].replace("." , "") #Accedo al valor pero tengo que limpiar datos para volverlo float 
#locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') solo para . y ,  pero en entero
locale.setlocale(locale.LC_ALL, 'nl_NL') #valor str convertido a float 
valuo_gob= locale.atof(sal_entid)
print(type(valuo_gob), valuo_gob ) # Pendiente convertirlo en numero

#--------------------------------------
#Saldo técnico a favor del responsable
saldo_respon = f_Arg[0].iloc[[42],[3]] #Saldo de impuesto a favor de AFIP
sal_respon = saldo_respon.values[0][0]  #Accedo al valor pero tengo que limpiar datos para volverlo float 
locale.setlocale(locale.LC_ALL, 'nl_NL') #valor str convertido a float 
valuo_respon= locale.atof(sal_respon)
print(type(valuo_respon), valuo_respon ) # Pendiente convertirlo en numero


<class 'float'> 3313414.59
<class 'float'> 0.0


##### c) Fomulario de Colombia con dos divisiones 
Debo de importar libreria de pandas 
from seaborn "import load_dataset"

In [4]:
#Lectura accede a la primera y segunda sección 
for fila in f_Col_300[0].itertuples():#salo las primeras 11 filas
    print(fila)
#print(f_Col_300)



Pandas(Index=0, _1=nan, _2=nan, _3=nan, _4=nan, _5=nan, Liquidación='Comprasprivada PRIVADA', _7='Ingresos', _8=nan, _9=nan)
Pandas(Index=1, _1=nan, _2=nan, _3=nan, _4=nan, _5=nan, Liquidación='Impuesto', _7=nan, _8=nan, _9=nan)
Pandas(Index=2, _1=nan, _2=nan, _3=nan, _4=nan, _5=nan, Liquidación='ImportaciónGenerado Nacionales', _7=nan, _8='300', _9=nan)
Pandas(Index=3, _1='1. Año 2', _2=0.0, _3='2', _4=2.0, _5=nan, Liquidación='3. Período 2', _7=nan, _8=nan, _9=nan)
Pandas(Index=4, _1='Espacio reservado para la DIAN', _2=nan, _3=nan, _4=nan, _5=nan, Liquidación='4. Número de formulario', _7=nan, _8='3004648676658', _9=nan)
Pandas(Index=5, _1='5. Número de Identificación Tributaria (NIT)', _2=nan, _3=nan, _4=nan, _5=nan, Liquidación='6. DV 7. Primer apellido 8. Segundo apellido 9. Primer nombre', _7=nan, _8='10. Otros nombres', _9=nan)
Pandas(Index=6, _1=nan, _2=nan, _3=nan, _4=9.0, _5='0 0 2 1 8', Liquidación='5 7 8 7', _7=nan, _8=nan, _9=nan)
Pandas(Index=7, _1='12.Cód. Dirección 11.

In [26]:
razon_soc1 = f_Col_300[0].iloc[9,[0]].values # linea 9 col 1

period= f_Col_300[0].iloc[3,[5]].values[0]
reg_per = re.compile(r'Período [1-6]{1}')
perio_f= reg_per.search(period)
valor = perio_f.group(0).split(' ')
valor_periodo = valor[1] # valor periodo asegurado
cod_verif = f_Col_300[0].iloc[4,[7]].values 
año =  f_Col_300[0].iloc[3,[0,1,2,3]].values  # linea verlo de otra forma sacar el año tiene str y  int
saldo_pagar= f_Col_300[0].iloc[52,[7]]
saldo_favor= f_Col_300[0].iloc[54,[8]]

print(valor_periodo, razon_soc1, año, saldo_pagar)
print(saldo_favor)




2 ['SISTEMAS COLOMBIA S A S'] ['1. Año 2' 0.0 '2' 2.0] Unnamed: 6    666,868,000
Name: 52, dtype: object
Unnamed: 7    0.0
Name: 54, dtype: object


In [71]:
import pdfplumber
import re
ruta_col = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\Decla. IVA II BIM 2022 - CO10.PDF"

with pdfplumber.open(ruta_col) as pdf:
    pag = pdf.pages[0]
    text = pag.extract_text()
form= text.split('\n')[0]
form_n= text.split('\n')[3]
nombre = text.split('\n')[4]

   

# print( form, form_n)
# print(text)
#Renglon  y valor
# for m in re.finditer(r'[0-9]{2}\s(\d{1,3}[,]?)+', text):
#     print(m.group(0))

# for m in re.finditer(r'[0-9]{2}\s(\d{1,3}[,]?)+', text):
#     print(m.group(0))



# Lectura de formulario descripción, renglon y valor
for m in re.finditer(r'.+?\s\d{2}\s(\d{1,3}[,]?){1,}', text):
    print(m.group(0))

Por operaciones gravadas al 5% 27 0
 o Eimnp voertnatdaa cerveza de producción nacional o  62 0
Por operaciones gravadas a la tarifa general 28 11,143,096,000
 ad En venta de gaseosas y similares 63 0
A(b.aI.sUe p gorra ovapbelrea ceisopneecsi aglr)avadas                                29 0
 Gen Esinm vilaernetas  5d%e licores, aperitivos, vinos y  64 0
Por exportación de bienes 30 0
 sto  Econn rseutimroo d, em uinevsetnratasr gior aptaisr ao  adcotnivaocsi ofinjoess,  65 0
Por exportación de servicios 31 161,460,966,000
 pue IaVnAu lraedcausp, erreasdcoin ednid adse voo rluecsiuoenletass en compras  66 0
Pinoter rvneanctiaosn aal sociedades de comercialización  32 0
 I Tgoratavla idmapsuesto generado por operaciones  67 2,125,572,000
Por ventas a zonas francas 33 0
 Por importaciones gravadas a tarifa del 5% 68 0
s Por juegos de suerte y azar 34 0
 Por importaciones gravadas la tarifa general 69 0
Por venta de gaseosas y similares 37 0
 ble Pgeonr ecroamlpras de bienes gravados a tar

In [65]:

# Obtener y limpiar datos e una linea
#a = re.compile(r'1. Año\s([0-9]\s){4}') 
#b = a.match(text)
b= re.search(r'Año\s([0-9]\s){4}', text)
año = re.search(r'([0-9]\s){4}',b.group(0)) 
periodo = re.search(r'([1-6]\s){1}',b.group(0)) 
print(año.group(0).replace(" ", ""), periodo.group(0).replace(" ", ""))



2022 2


In [ ]:
from decimal import *

Context(2,2,)

print(Decimal("666,868,000", context={ }))